K-Means Clustering with Toronto Neighborhoods 

Begin by assigning beautiful soup object to page contents of wikipedia page

In [2]:
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

create list of table contents by parsing table out of page contents while excluding rows with unassigned Borough

In [3]:
import pandas as pd

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


create pandas dataframe from list and rename boroughs.

In [4]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

print(df)

    PostalCode                Borough  \
0          M3A             North York   
1          M4A             North York   
2          M5A       Downtown Toronto   
3          M6A             North York   
4          M7A           Queen's Park   
..         ...                    ...   
98         M8X              Etobicoke   
99         M4Y       Downtown Toronto   
100        M7Y  East Toronto Business   
101        M8Y              Etobicoke   
102        M8Z              Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4                        Ontario Provincial Government  
..                                                 ...  
98       The Kingsway, Montgomery Road, Old Mill North  
99                                Church and Welles

Append spatial coordinates for each postal code

In [5]:
coords_df = pd.read_csv(r'C:\Users\jackh\Desktop\projects\IBM-Applied-Data-Science-Capstone\data\Geospatial_Coordinates.csv')
toronto_df = pd.merge(left=df, right=coords_df, left_on='PostalCode', right_on='Postal Code').drop('Postal Code', axis=1)
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


Visualize Neighborhoods

In [6]:
print('Our Toronto DataFrame has {} Boroughs and {} Neighborhoods.'.format(
    len(toronto_df['Borough'].unique()),
    len(toronto_df['Neighborhood'].unique())
))

Our Toronto DataFrame has 15 Boroughs and 103 Neighborhoods.


Find coordinates for city of Toronto to define map view

In [7]:
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'Latitude: {latitude}, Longitude: {longitude}')

Latitude: 43.6534817, Longitude: -79.3839347


Initialize map and add markers for each neighborhood

In [ ]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)

map_toronto

Filter dataset to only Downtown Toronto

In [9]:
toronto_df['Count'] = 1
dt_toronto_df = toronto_df[toronto_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dt_toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Count
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1


In [ ]:
map_dt_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(dt_toronto_df['Latitude'], dt_toronto_df['Longitude'], dt_toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fil=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_dt_toronto)

map_dt_toronto

Define Foursquare API credentials to get venue and location data

In [11]:
CLIENT_ID = 'IOSOZCWY3TE0ZFLCXGC3NDBVYPRM4CLR2SGZUDSXFNHI2S1B' # your Foursquare ID
CLIENT_SECRET = '4Y4F3QXQ0CA2VSQLL2OLOVXLDRHRSTXKMUD04RQPZKTJXO1N' # your Foursquare Secret
ACCESS_TOKEN = 'SS0UU3CPORXILGWUOB1WNAKWEW0HS2BCIWU1SN4KXHHND1L5' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100

First, I will explore the first neighborhood in the dataset, "Regent Park, Harbourfront"

In [12]:
dt_toronto_df.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [13]:
neighborhood_latitude = dt_toronto_df.loc[0, 'Latitude']
neighborhood_longitude = dt_toronto_df.loc[0, 'Longitude']

neighborhood_name = dt_toronto_df.loc[0, 'Neighborhood']

print('Latitude and logitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and logitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [17]:
import json

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-19-550f9251816b>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


In [21]:
first_record = dt_toronto_df.loc[dt_toronto_df['Neighborhood'].str.contains('Regent Park, Harbourfront')]
first_record

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Count
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,1


Now that we've seen results for one neighborhood. Let's get nearby venues for all neighborhoods in Downtown Toronto.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
dt_toronto_venues = getNearbyVenues(names=dt_toronto_df['Neighborhood'], latitudes=dt_toronto_df['Latitude'], longitudes=dt_toronto_df['Longitude'])
dt_toronto_venues

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
...,...,...,...,...,...,...,...
1116,Church and Wellesley,43.66586,-79.383160,Asahi Sushi,43.669874,-79.382943,Sushi Restaurant
1117,Church and Wellesley,43.66586,-79.383160,Town Inn Suites,43.669056,-79.382573,Hotel
1118,Church and Wellesley,43.66586,-79.383160,McDonald's,43.668854,-79.385962,Fast Food Restaurant
1119,Church and Wellesley,43.66586,-79.383160,Petro-Canada,43.666637,-79.378028,Gas Station


In [24]:
print(dt_toronto_venues.shape)
dt_toronto_venues.head()

(1121, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned per neighborhood

In [25]:
dt_toronto_venues.groupby('Neighborhood', as_index=False).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,60,60,60,60,60,60
1,"CN Tower, King and Spadina, Railway Lands, Har...",15,15,15,15,15,15
2,Central Bay Street,62,62,62,62,62,62
3,Christie,17,17,17,17,17,17
4,Church and Wellesley,78,78,78,78,78,78
5,"Commerce Court, Victoria Hotel",100,100,100,100,100,100
6,"First Canadian Place, Underground city",100,100,100,100,100,100
7,"Garden District, Ryerson",100,100,100,100,100,100
8,"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100
9,"Kensington Market, Chinatown, Grange Park",69,69,69,69,69,69


Now, let's check how many unique venue categories were returned

In [105]:
print('There are {} unique categories.'.format(len(dt_toronto_venues['Venue Category'].unique())))

There are 206 unique categories.


We have our Venue data for all neighborhoods. Finally, let's begin analysis.

In [26]:
# one hot encoding
dt_toronto_onehot = pd.get_dummies(dt_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_toronto_onehot['Neighborhood'] = dt_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_toronto_onehot.columns[-1]] + list(dt_toronto_onehot.columns[:-1])
dt_toronto_onehot = dt_toronto_onehot[fixed_columns]

dt_toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
dt_toronto_onehot.shape

(1121, 208)

Next, let's group rows by neighborhood and by taking the mean frequency of the frequency of occurrence of each category

In [28]:
dt_toronto_grouped = dt_toronto_onehot.groupby('Neighborhood').mean().reset_index()
dt_toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.016667,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.016129,0.000000,0.000000,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.025641,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,...,0.012821,0.012821,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.010000,0.000000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.000000,0.01,0.00,0.00,0.000000,0.000000,0.010000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.014493,0.00,0.00,0.00,0.043478,0.000000,0.057971,0.014493


In [29]:
dt_toronto_grouped.shape

(17, 208)

Let's print the top 5 most frequented venues for each neighborhood.

In [30]:
num_top_venues = 5

for hood in dt_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_toronto_grouped[dt_toronto_grouped["Neighborhood"] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2              Bakery  0.05
3  Seafood Restaurant  0.03
4      Farmers Market  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.13
1  Airport Terminal  0.13
2   Harbor / Marina  0.07
3       Coffee Shop  0.07
4     Boat or Ferry  0.07


----Central Bay Street----
              venue  freq
0       Coffee Shop  0.18
1              Café  0.06
2    Sandwich Place  0.05
3  Department Store  0.03
4   Bubble Tea Shop  0.03


----Christie----
                venue  freq
0       Grocery Store  0.24
1                Café  0.18
2                Park  0.12
3  Athletics & Sports  0.06
4          Baby Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.06
3              Gay B

Now we will create a new datafram with 10 most common venues for each neighborhood.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_toronto_grouped['Neighborhood']

for ind in np.arange(dt_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Restaurant,Beer Bar,Indian Restaurant,Department Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boat or Ferry,Sculpture Garden,Bar,Rental Car Location,Boutique,Airport Lounge
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Department Store,Bubble Tea Shop,Bank,Burger Joint,Salad Place,Spa,Restaurant
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Baby Store,Candy Store,Restaurant,Italian Restaurant,Nightclub,Bank
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Pub,Café,Mediterranean Restaurant,Men's Store,Yoga Studio


Clustering Neighborhoods Based on Venue Categories

In [33]:
from sklearn.cluster import KMeans

kclusters = 7

dt_toronto_grouped_clustering = dt_toronto_grouped.drop('Neighborhood', 1)
print(dt_toronto_grouped_clustering.head())
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

   Yoga Studio  Afghan Restaurant   Airport  Airport Food Court  \
0     0.000000           0.000000  0.000000            0.000000   
1     0.000000           0.000000  0.066667            0.066667   
2     0.016129           0.000000  0.000000            0.000000   
3     0.000000           0.000000  0.000000            0.000000   
4     0.025641           0.012821  0.000000            0.000000   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0        0.000000         0.000000          0.000000             0.000000   
1        0.066667         0.133333          0.133333             0.000000   
2        0.000000         0.000000          0.000000             0.000000   
3        0.000000         0.000000          0.000000             0.000000   
4        0.000000         0.000000          0.000000             0.012821   

   Antique Shop  Aquarium  ...   Theater  Theme Restaurant  \
0           0.0       0.0  ...  0.000000          0.000000   
1         

array([5, 2, 0, 3, 0, 0, 0, 5, 0, 5])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [114]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_toronto_merged = dt_toronto_df

# merge dt_toronto_grouped with dt_toronto_df to add latitude/longitude for each neighborhood
dt_toronto_merged = dt_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dt_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Count,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,6,Coffee Shop,Pub,Park,Bakery,Theater,Café,Yoga Studio,Farmers Market,Chocolate Shop,Dessert Shop
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Cosmetics Shop,Japanese Restaurant,Pizza Place,Diner,Italian Restaurant,Lingerie Store
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,0,Coffee Shop,Café,Restaurant,Clothing Store,Bakery,Cosmetics Shop,Cocktail Bar,American Restaurant,Moroccan Restaurant,Seafood Restaurant
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,0,Bakery,Cocktail Bar,Coffee Shop,Seafood Restaurant,Pharmacy,Pub,Beer Bar,Farmers Market,Restaurant,Cheese Shop
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,2,Coffee Shop,Café,Sandwich Place,Restaurant,Italian Restaurant,Bubble Tea Shop,Salad Place,Japanese Restaurant,Burger Joint,Spa


Finally, let's visualize our clusters!

In [115]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_toronto_merged['Latitude'], dt_toronto_merged['Longitude'], dt_toronto_merged['Neighborhood'], dt_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters